<a href="https://colab.research.google.com/github/karinamsilva/100-Days-of-Swift/blob/main/Rentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
#Carregamento de dados

df_carbon_offsets = pd.read_csv('/content/carbon_offsets.csv')
df_customers = pd.read_csv('/content/customers_data.csv')
df_rentals = pd.read_json('/content/rentals_data.json')
df_vehhicles = pd.read_json('/content/vehicles_data.json')

Identificação de dados

- Datas
- Valores Negativos
- DataType

Correções

In [ ]:
#Pré analise
# Comandos importantes - info()/head()/columns()

df_carbon_offsets.info()
df_carbon_offsets.head()
df_customers.info()
df_customers.head()
df_rentals.info()
df_rentals.head()
df_vehhicles.info()
df_vehhicles.head()
#df_carbon_offsets.columns


In [ ]:
#Customers - Valor ausente
df_media_age = df_customers['age'].mean()
df_customers['age'] = df_customers['age'].fillna(df_media_age)

In [ ]:
#Daily rate negativo

df_rentals.loc[df_rentals['daily_rate'] <= 0, 'daily_rate'] = pd.NA
display(df_rentals)

#Caso de usar a média nesse campo
#transformar de string para numero

df_rentals['daily_rate'] = pd.to_numeric(df_rentals['daily_rate'], errors='coerce')
df_media_daily_rate = df_rentals['daily_rate'].mean()
df_rentals['daily_rate'] = df_rentals['daily_rate'].fillna(df_media_daily_rate)



In [ ]:
#Vehicles - c02_g_per km ausente (Média ou mesmo valor do eco)

co2_mean = df_vehhicles['co2_g_per_km'].mean()
df_vehhicles['co2_g_per_km'] = df_vehhicles['co2_g_per_km'].fillna(co2_mean)
display(df_vehhicles)


In [ ]:
#Rentals - data inválida

df_rentals['start_date'] = pd.to_datetime(df_rentals['start_date'], errors='coerce')
df_rentals['end_date'] = pd.to_datetime(df_rentals['end_date'], errors='coerce')
display(df_rentals)

Engenharia de Dados

- Calcule a duração de cada locação (em dias)
- Calcule a receita total por locação
- Faça o merge com a base de veiculos
- Calcule a emissão de C02 por locação
- Extraia ano e mês da data de inicio da locação

In [ ]:
#Criar backup de dados

df_carbon_clean = df_carbon_offsets
df_rentals_clean = df_rentals
df_customer_clean = df_customers
df_vehicles_clean = df_vehhicles


In [ ]:
#Duração de cada locação
df_rentals_clean['dias_locação'] = (df_rentals_clean['end_date'] - df_rentals_clean['start_date']).dt.days

display(df_rentals_clean)

In [ ]:
#Receita total por locação

df_rentals_clean['receita_total'] = (df_rentals_clean['dias_locação'] * df_rentals_clean['daily_rate'])



In [ ]:
#Merge com a base de veiculos

df_consolidado = df_rentals_clean.merge(df_vehicles_clean, on='vehicle_id')
df_consolidado = df_consolidado.merge(df_customer_clean, on='customer_id')
display(df_consolidado)


In [ ]:
#Emissão de co2 por locação

df_consolidado['emissão_co2'] = (df_consolidado['co2_g_per_km'] * df_consolidado['distance_km'])

display(df_consolidado)

In [78]:
#Extraia ano e mês da data de inicio de locação

df_consolidado['ano'] = df_rentals_clean['start_date'].dt.year
df_consolidado['mês'] = df_rentals_clean['start_date'].dt.month

display(df_consolidado)




,rental_id,customer_id,vehicle_id,start_date,end_date,city_x,daily_rate,distance_km,dias_locação,model,fuel_type,co2_g_per_km,category,customer_type,age,city_y,loyalty_score,emissão_co2,ano,mês
0,2001,C001,V100,2024-01-05,2024-01-08,São Paulo,180.0,320,3.0,HB20,Gasolina,120.0,Econômico,Recorrente,35.0,São Paulo,90,38400.0,2024.0,1.0
1,2002,C002,V101,NaT,NaT,Curitiba,150.0,210,NaN,Corolla,Gasolina,160.0,Sedan,Novo,28.0,Curitiba,60,33600.0,NaN,NaN
2,2003,C003,V102,NaT,2024-01-15,Salvador,220.0,0,NaN,Compass,Diesel,180.0,SUV,Novo,36.0,Salvador,50,0.0,NaN,NaN
3,2004,C004,V103,2024-02-01,2024-02-10,São Paulo,300.0,980,9.0,Tesla Model 3,Elétrico,0.0,Elétrico,Corporativo,45.0,São Paulo,95,0.0,2024.0,2.0
4,2005,C001,V104,2024-02-15,2024-02-16,Rio de Janeiro,212.5,120,1.0,Onix,Gasolina,115.0,Econômico,Recorrente,35.0,São Paulo,90,13800.0,2024.0,2.0
